In [4]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

USERNAME = "postgres"
PASSWORD = quote_plus("Rutgers123")   # safe even if password has special chars
HOST = "127.0.0.1"
PORT = 5401
DB = "postgres"

engine = create_engine(
    f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}",
    connect_args={"connect_timeout": 5}   # don't hang forever
)

df = pd.read_sql("SELECT * FROM customers", engine)
df

,customer_id,company_name,plan_tier,mrr,signup_date,churn_date,country,num_users,support_tickets,last_login_days_ago,nps_score
0,1001,GlobalSoft,Basic,49.0,2022-06-28,2023-06-20,France,1,8,45,-12
1,1002,WindDigital,Enterprise,799.0,2022-05-16,None,France,36,2,19,37
2,1003,WaveSync,Pro,299.0,2021-01-13,None,United States,26,1,19,69
3,1004,HiveAI,Pro,199.0,2021-07-25,None,United States,8,4,4,40
4,1005,UnionAI,Basic,79.0,2022-01-16,2022-04-11,United Kingdom,10,13,101,-46
...,...,...,...,...,...,...,...,...,...,...,...
4995,5996,ZenByte1,Enterprise,1199.0,2021-06-27,2022-05-18,France,199,0,66,39
4996,5997,XcelForce7,Basic,79.0,2022-11-08,2023-06-17,France,4,18,55,37
4997,5998,ZoomSystems32,Basic,79.0,2023-01-23,2024-08-06,United States,3,0,79,44
4998,5999,VibeEdge61,Enterprise,799.0,2023-11-18,None,India,33,3,13,22


In [5]:
df.shape, df.isna().sum()

((5000, 11),
 customer_id               0
 company_name              0
 plan_tier                 0
 mrr                       0
 signup_date               0
 churn_date             3566
 country                   0
 num_users                 0
 support_tickets           0
 last_login_days_ago       0
 nps_score                 0
 dtype: int64)

In [6]:
import pandas as pd

df["signup_date"] = pd.to_datetime(df["signup_date"])
df["churn_date"] = pd.to_datetime(df["churn_date"], errors="coerce")

df["is_churned"] = df["churn_date"].notna().astype(int)

end_date = df["churn_date"].fillna(pd.Timestamp.today().normalize())
df["tenure_days"] = (end_date - df["signup_date"]).dt.days.clip(lower=0)

df["clv"] = (df["tenure_days"] / 30.0) * df["mrr"]

df[["is_churned", "tenure_days", "clv"]].describe()

,is_churned,tenure_days,clv
count,5000.000000,5000.000000,5000.000000
mean,0.286800,1032.370800,10317.860707
std,0.452313,546.730042,14994.617697
min,0.000000,30.000000,39.000000
25%,0.000000,519.750000,1396.675000
50%,0.000000,1107.000000,3035.633333
75%,1.000000,1479.000000,11055.600000
max,1.000000,1879.000000,93487.633333


In [9]:
from scipy import stats

churned = df.loc[df["is_churned"] == 1, "support_tickets"]
retained = df.loc[df["is_churned"] == 0, "support_tickets"]

t_stat, p_value = stats.ttest_ind(churned, retained, equal_var=False)
if p_value < 0.001:
    p_display = "p < 0.001"
else:
    p_display = f"p = {p_value:.3f}"

print(f"T-statistic: {t_stat:.3f}")
print(f"P-value: {p_display}")

print("Mean support tickets (churned):", churned.mean())
print("Mean support tickets (retained):", retained.mean())

T-statistic: 11.975
P-value: p < 0.001
Mean support tickets (churned): 5.819386331938633
Mean support tickets (retained): 4.054683118339876


## Inference: Support Tickets vs Churn (Statistical & Business Interpretation)

We conducted an independent two-sample t-test (Welch’s t-test) to evaluate whether the average number of support tickets differs between churned and retained customers.

### Statistical Results
- T-statistic: 11.975
- P-value: < 0.001

### Group Means
- Churned customers: 5.82 tickets (avg)
- Retained customers: 4.05 tickets (avg)
- Absolute difference: ~1.77 additional tickets

Because the p-value is < 0.001, we reject the null hypothesis of equal means.  
There is strong statistical evidence that churned customers submit significantly more support tickets than retained customers.

### Effect Interpretation
The magnitude of the difference (~1.77 tickets) is practically meaningful, not just statistically significant. This suggests that higher support interaction volume may reflect friction, dissatisfaction, or product usability issues.

### Business Implication
Support ticket frequency can serve as an early churn risk indicator.

Potential actions:
- Trigger proactive outreach after a threshold number of tickets
- Escalate repeat-ticket customers to customer success teams
- Monitor unresolved ticket duration as an additional churn signal

This variable should be considered a strong behavioral predictor in the churn model.

In [10]:
contingency = pd.crosstab(df["plan_tier"], df["is_churned"])
chi2, p, dof, expected = stats.chi2_contingency(contingency)

print(contingency)
print(f"Chi2: {chi2:.3f}, dof: {dof}, P-value: {p:.4f}")

is_churned     0    1
plan_tier            
Basic       1549  951
Enterprise   856  143
Pro         1161  340
Chi2: 234.539, dof: 2, P-value: 0.0000


## Inference: Plan Tier vs Churn

A Chi-square test of independence was conducted to examine the relationship between plan tier and churn status.

- Chi-square statistic: 234.539  
- Degrees of freedom: 2  
- P-value: < 0.001  

Since the p-value is < 0.001, we reject the null hypothesis of independence.

Conclusion:
There is a statistically significant association between plan tier and churn. Churn behavior differs meaningfully across Basic, Pro, and Enterprise plans.

In [11]:
(df.groupby("plan_tier")["is_churned"].mean().mul(100).round(2)).sort_values(ascending=False)

plan_tier
Basic         38.04
Pro           22.65
Enterprise    14.31
Name: is_churned, dtype: float64

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

features = ['mrr', 'num_users', 'support_tickets', 'nps_score']
# Exclude last_login_days_ago
X = df[features]
y = df["is_churned"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

model = LogisticRegression(max_iter=2000, class_weight="balanced")
model.fit(X_train, y_train)

probs = model.predict_proba(X_test)[:, 1]
preds = (probs >= 0.5).astype(int)

print("ROC AUC:", round(roc_auc_score(y_test, probs), 3))
print(classification_report(y_test, preds))

ROC AUC: 0.665
              precision    recall  f1-score   support

           0       0.81      0.58      0.67       892
           1       0.39      0.67      0.49       358

    accuracy                           0.60      1250
   macro avg       0.60      0.62      0.58      1250
weighted avg       0.69      0.60      0.62      1250



## Model Performance: Logistic Regression

- ROC AUC: 0.665  
- Accuracy: 60%

Class-wise performance:
- Non-churn (0): Precision = 0.81, Recall = 0.58
- Churn (1): Precision = 0.39, Recall = 0.67

Interpretation:
The model shows moderate discriminative ability (AUC = 0.665).  
It performs better at identifying churn cases (recall = 0.67) than at precisely predicting them (precision = 0.39), meaning it captures most churners but includes false positives.

Conclusion:
The model provides useful signal but would benefit from further feature engineering, threshold tuning, or more advanced models to improve precision and overall performance.
Additionally, lower NPS scores are strongly associated with higher churn risk, suggesting customer satisfaction is a key retention driver.

In [19]:
df["churn_probability"] = model.predict_proba(X)[:, 1]

high_risk = df[df["churn_probability"] > 0.7]
revenue_at_risk = high_risk["mrr"].sum()

print(f"High risk customers: {len(high_risk)}")
print(f"MRR at risk: ${revenue_at_risk:,.0f}")

high_risk[["customer_id","company_name","plan_tier","mrr","support_tickets","last_login_days_ago","nps_score","churn_probability"]].head(10)

High risk customers: 153
MRR at risk: $7,607


,customer_id,company_name,plan_tier,mrr,support_tickets,last_login_days_ago,nps_score,churn_probability
4,1005,UnionAI,Basic,79.0,13,101,-46,0.724839
23,1024,EdgeDigital,Basic,59.0,8,38,-80,0.725035
41,1042,AlphaCloud,Basic,49.0,25,31,23,0.753319
85,1086,CloudSystems,Basic,49.0,10,36,-56,0.714820
102,1103,MeshIO,Basic,49.0,14,43,-46,0.740060
106,1107,KiteAnalytics,Basic,49.0,14,45,-18,0.701914
130,1131,FastStudio,Basic,49.0,22,35,27,0.719867
138,1139,IntelHub,Basic,59.0,24,35,34,0.728174
149,1150,FluxSolutions,Basic,39.0,17,1,-5,0.715755
205,1206,CoreNet,Basic,49.0,9,28,-64,0.715992


## High-Risk Customer Identification & Revenue Impact

Using the trained model and a probability threshold of 0.7:

- High-risk customers identified: 153
- Monthly Recurring Revenue (MRR) at risk: $7,607

Interpretation:
The model flags 153 customers as likely to churn, representing $7.6K in potential monthly revenue loss.

Business Implication:
Targeted retention efforts (proactive outreach, account review, support escalation) focused on these high-risk customers could directly protect recurring revenue and improve customer lifetime value.

In [20]:
import pandas as pd

coef_df = pd.DataFrame({
    "feature": features,
    "coef": model.coef_[0]
}).sort_values("coef", ascending=False)

coef_df

,feature,coef
2,support_tickets,0.048883
1,num_users,-0.000551
0,mrr,-0.000832
3,nps_score,-0.006761


In [24]:
import numpy as np

# Make sure dates are datetime
df["signup_date"] = pd.to_datetime(df["signup_date"])
df["churn_date"] = pd.to_datetime(df["churn_date"], errors="coerce")

# Basic flags
df["is_churned"] = df["churn_date"].notna().astype(int)

# Risk segment based on churn probability
df['risk_segment'] = pd.qcut(df['churn_probability'],
                              q=[0, 0.60, 0.85, 1.0],
                              labels=['Low', 'Medium', 'High'])

# Export set for Tableau
export_df = df[[
    "customer_id",
    "company_name",
    "plan_tier",
    "country",
    "mrr",
    "num_users",
    "support_tickets",
    "last_login_days_ago",
    "nps_score",
    "signup_date",
    "churn_date",
    "is_churned",
    "churn_probability",
    "risk_segment"
]].copy()

export_df.to_csv("saas_customer_scoring.csv", index=False)
print("Saved: saas_customer_scoring.csv", export_df.shape)

Saved: saas_customer_scoring.csv (5000, 14)


In [25]:
import pandas as pd
import numpy as np

cohort_df = df[["customer_id", "signup_date", "churn_date"]].copy()
cohort_df["signup_month"] = cohort_df["signup_date"].dt.to_period("M").dt.to_timestamp()

# Define an "end date" (for active users, use today)
today = pd.Timestamp.today().normalize()
cohort_df["end_date"] = cohort_df["churn_date"].fillna(today)

# months active (integer)
cohort_df["months_active"] = (
    (cohort_df["end_date"].dt.year - cohort_df["signup_date"].dt.year) * 12
    + (cohort_df["end_date"].dt.month - cohort_df["signup_date"].dt.month)
).clip(lower=0)

# Expand each customer into one row per month they stayed active
rows = []
for _, r in cohort_df.iterrows():
    for m in range(int(r["months_active"]) + 1):
        rows.append((r["signup_month"], m, r["customer_id"]))

retention_long = pd.DataFrame(rows, columns=["cohort_month", "months_since_signup", "customer_id"])

# cohort sizes
cohort_sizes = retention_long.groupby("cohort_month")["customer_id"].nunique().reset_index(name="cohort_size")

# retained count by cohort + month index
retained = retention_long.groupby(["cohort_month", "months_since_signup"])["customer_id"].nunique().reset_index(name="customers_retained")

# retention %
retention_curve = retained.merge(cohort_sizes, on="cohort_month")
retention_curve["retention_pct"] = (retention_curve["customers_retained"] / retention_curve["cohort_size"] * 100).round(2)

retention_curve.to_csv("saas_cohort_retention_curve.csv", index=False)
print("Saved: saas_cohort_retention_curve.csv", retention_curve.shape)

Saved: saas_cohort_retention_curve.csv (1602, 5)
